In [1]:
import os
import faiss
import tqdm
import json
import numpy as np
from utils import extract_features_single, load_pca, create_model_and_tokenizer

from datasets import load_dataset

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model, tokenizer = create_model_and_tokenizer(model_name)
model = model.cuda()

query_data = load_dataset('json', data_files='./lm-eval-train.jsonl', split='train')

data = load_dataset('json', data_files='/root/data/wiki_en.jsonl', split='train')


Loading tokenizer...

Loading model...

In [2]:
vecs = np.load(f'./features/sentence-transformers/all-MiniLM-L6-v2/wiki_en.npy')

In [3]:
def build_index(
    xb: np.ndarray,
    d: int = None,
):
    if d is None:
        d = xb.shape[-1]
    res = faiss.StandardGpuResources()
    index = faiss.index_factory(d, "IVF100,PQ8")
    # Sample 1_000_000 vectors to train the index.
    # xt = xb[np.random.choice(xb.shape[0], 1000_000, replace=False)]
    xt = xb
    index.train(xt)
    # index.add(xb)
    return index

def add_to_index(
    xb,
    index,
):
    index.add(xb)
    return index

In [4]:
def search(queries, k=100):
    features = extract_features_single(queries, model, tokenizer)
    _, ids = index.search(features, k=k)
    return [data[int(i)] for i in ids[0]]

In [5]:
index = build_index(vecs)

In [6]:
index = add_to_index(vecs, index)

In [ ]:
with open(f'/root/ft_data/lm_aug_wiki_all.jsonl', 'w') as f:
    for query in tqdm.tqdm(query_data):
        items = search(query['text'])
        for rank, item in enumerate(items):
            item['meta']['augment'] = {
                'rank': rank,
                'query_meta': query['meta'],
            }
            item['meta'].pop('timestamp')
            f.write(json.dumps(item) + '\n')

 16%|█████████████████████▊                                                                                                                 | 12825/79195 [05:05<24:25, 45.30it/s]

In [ ]:
exit()